## Import the pymongo package and connect to the cluster

In [ ]:
import pymongo
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
print(client)

## Get a reference to a database with either of these syntax options

<p>
<details><summary>Click for <b>java</b></summary>
<p>

```
MongoDatabase db = mongoClient.getDatabase("productdb");
```
</p>
</details>
<p>
</details>


In [ ]:
# Property notation
db = client.productdb
print(db)

# Indexer notation
db = client['productdb']
print(db)

# Indexer notation with a variable
dbname = 'productdb'
db = client[dbname]
print(db)

# Method call
pr = client.get_database('productdb')
print(pr)

## Could use annotations to help show the data types. Annotations in Python give type hints that linters can use to try to identify potential bugs. It's not a strict as static typing but can be helpful and is also useful for prototyping in Python before converting to Java or C#.

In [ ]:
from pymongo.database import Database
dbname: str = 'productdb'
db: Database = client.get_database(dbname) 
print(db)
print(type(db))

## Show the collections in the database
<p>
<details><summary>Click for <b>javascript</b></summary>
<p>

```
db.product.getCollectionNames()
```
</p>
</details>
<p>
</details>

In [ ]:
db.list_collection_names()

# This is deprecated
#db.collection_names()

## Let's get rid of the collection and see how it is dynamically created when we first use it

In [ ]:
db.products.drop()

In [ ]:
print(db.list_collection_names())

## The `insert` method still works but is deprecated and should be avoided
<p>
<details><summary>Click for <b>javascript</b></summary>
<p>

```
db.products.insert({manufacturer: "KitchenAid", model:1})
db.products.find({manufacturer: "KitchenAid"}```
</p>
</details>


In [ ]:
# insert works but is deprecated
db.products.insert({"manufacturer":"KitchenAid", "model":1})

# find returns a cursor object, not the actual data
print(db.products.find({"manufacturer":"KitchenAid"}))

# fetch all the data by casting it to a list
list(db.products.find({"manufacturer":"KitchenAid"}))

## Use the newer methods `insert_one` and `insert_many`

<p>
<details><summary>Click for <b>java</b></summary>
<p>

```
MongoCollection<Document> products = db.getCollection("products");

// insert one
products.insertOne(new Document("_id", new ObjectId()).append("manufacturer", "GE").append("model",2));

// insert many
List<Document> newproducts = new ArrayList<>();
newproducts.add(new Document("manufacturer", "KitchenAid").append("model":3));
newproducts.add(new Document("manufacturer", "Samsung").append("model":4));

products.insertMany(newproducts, new InsertManyOptions().ordered(false));
 
Document product1 = products.find(new Document("manufacturer", "KitchenAid")).first();
System.out.println(product1.toJson());

// find one document with Filters.eq()
Document product2 = products.find(eq("Manufacturer", "KitchenAid")).first();
System.out.println(product2.toJson());
```
</p>
</details>

In [ ]:
# Insert a single document
db.products.insert_one({"manufacturer":"GE", 'model':2})

# Bulk insert many documents by providing a list
db.products.insert_many([{"manufacturer":"KitchenAid", 'model':3}, 
                         {"manufacturer":"Samsung", 'model':4}])

list(db.products.find({"manufacturer":"KitchenAid"}))

## The `find_one` function will find and return a Python dictionary of the requested document. If more than one match, it will return the first one based on the method it used to find it, such as whether it used an index, etc.

In [ ]:
products = db.products.find({"manufacturer":"KitchenAid"})
for p in products:
    print(p)

## returns a single document not a collection of multiple documents

In [ ]:
db.products.find_one()

## Fetch the single document into a variable to work with it at the programming level in an optimistic locking style

In [ ]:
product = db.products.find_one({"manufacturer":"KitchenAid"})
product_id = product['_id']
manufacturer = product['manufacturer']
print(product_id, manufacturer)

## Fetch a specific document by its `_id` property

In [ ]:
# need to import the ObjectId function
from bson.objectid import ObjectId
db.products.find_one({"_id": ObjectId(product_id)})

## The `find` function returns a cursor object, which you can iterate through with a loop or materialize by converting to a `list`

In [ ]:
db.products.find()

In [ ]:
list(db.products.find())

In [ ]:
for product in db.products.find():
    print(product)

## You need to quote the special operators like `$gt`

In [ ]:
list(db.products.find({"model": {"$gt" : 2}}))

## There are several methods for counting documents

In [ ]:
# db.products.count() #deprecated
print(db.products.estimated_document_count())
print(db.products.count_documents({'manufacturer':'KitchenAid', "model": {"$gt":2}}))


## `delete` has a function for one or many, similar to `insert`

In [ ]:
db.products.delete_one({"manufacturer":"KitchenAid"})
list(db.products.find({"manufacturer":"KitchenAid"}))

## And of course updating

In [ ]:
# get rid of all the KitcheAid
db.products.delete_many({"manufacturer":"KitchenAid"})

# put some data in to update
db.products.insert_many([{"manufacturer":"KitchenAid", 'model':5}, 
                         {"manufacturer":"KitchenAid", 'model':6}])

db.products.update_one({"manufacturer":"KitchenAid"}, {"$set": {"price":100}})

for p in db.products.find({"manufacturer":"KitchenAid"}):
    print(p)

print('-' * 20)
db.products.update_many({"manufacturer":"KitchenAid"}, {"$set": {"price":200}})
for p in db.products.find({"manufacturer":"KitchenAid"}):
    print(p)


## `upsert` is more Pythonic as an optional parameter is used instead of another dictionary in the third position

In [ ]:
# Product 7 won't be added because upsert = False
db.products.update_one({"manufacturer":"KitchenAid", "model":7}, {"$set": {"price":100}}
                       , upsert = False)
for p in db.products.find({"manufacturer":"KitchenAid"}):
    print(p)

print('-' * 20)

# Product 8 will be added because upsert = True
db.products.update_one({"manufacturer":"KitchenAid", "model":8}, {"$set": {"price":100}}
                       , upsert = True)
for p in db.products.find({"manufacturer":"KitchenAid"}):
    print(p)


## `replace_one` will use the search condition to find the document and replace it with new values, but preserving the `_id`

In [ ]:
# fix model 8 in case it is wrong
db.products.update_one({"manufacturer":"KitchenAid", "model":8}, {"$set": {"price":100}}
                       , upsert = True)
for p in db.products.find({"manufacturer":"KitchenAid"}):
    print(p)

print('-' * 20)

product = db.products.find_one({"manufacturer":"KitchenAid", "model":8})
product['model'] = 9
product['price'] = 150
product['cost'] = 100

db.products.replace_one({"_id": ObjectId(product["_id"])}
                        , product)

for p in db.products.find({"manufacturer":"KitchenAid"}):
    print(p)


# Note how the price disappeared; it's usually safer to use update methods than replace


## Typically with an optimistic locking scheme, we would fetch one or more documents, modify them programmatically and then save them back to the collection.

In [ ]:
x = db.products.find_one({"model":9})
print(x, type(x))
x['price'] = 199
x['colors'] = ['white', 'black']
db.products.replace_one({"model":9}, x)
print(db.products.find_one({"model":9}))

## The `find_one_and_delete` function will return the original object in addition to deleting it

<p>
<details><summary>Click for <b>javascript</b></summary>
<p>

```
p1 = db.products.findOneAndDelete({"manufacturer":"KitchenAid", "model":9})
```
</p>
</details>
<p>
<details><summary>Click for <b>java</b></summary>
<p>

```
Document p1 = db.products.findOneAndDelete(new BsonDocument.parse("{'manufacturer': 'KitchenAid', 'model':9}"));
```
</p>
</details>


In [ ]:
p1 = db.products.find_one_and_delete({"manufacturer":"KitchenAid", "model":9})
print(p1)

print('-' * 20)

for p in db.products.find({"manufacturer":"KitchenAid"}):
    print(p)



## The `find_one_and_update` function will return the original object in addition to updating it
<p>
<details><summary>Click for <b>javascript</b></summary>
<p>

```
p1 = db.products.findOneAndUpdate({"manufacturer":"KitchenAid", "model":9}, {"$set":{"price":500}})
```
</p>
</details>
<p>
<details><summary>Click for <b>java</b></summary>
<p>

```
Document p1 = db.products.findOneAndUpdate(
      new BsonDocument.parse("{'manufacturer': 'KitchenAid', 'model':5}")
    , new BsonDocument("$set", new BSonDocument("price",500) ) 
    );
```
</p>
</details>



In [ ]:
p1 = db.products.find_one_and_update({"manufacturer":"KitchenAid", "model":5}, 
                                     {"$set":{"price":500}})
print(p1)

print('-' * 20)

for p in db.products.find({"manufacturer":"KitchenAid"}):
    print(p)


## The `find_one_and_replace` function will return the original object in addition to replacing it

<p>
<details><summary>Click for <b>javascript</b></summary>
<p>

```
p1 = db.products.findOneAndReplace({"manufacturer":"KitchenAid", "model":9}
                                 , {"manufacturer":"KitchenAid", "model":8}})
```
</p>
</details>
<p>
<details><summary>Click for <b>java</b></summary>
<p>

```
Document p1 = db.products.findOneAndReplace(
      new BsonDocument("manufacturer","KitchenAid").append("model", 9)
    , new BsonDocument("manufacturer","KitchenAid").append("model", 8)
    );
```
</p>
</details>



In [ ]:
p1 = db.products.find_one_and_replace({"manufacturer":"KitchenAid", "model":5}
                                      , {"manufacturer":"KitchenAid", "model":8})
print(p1)

print('-' * 20)

for p in db.products.find({"manufacturer":"KitchenAid"}):
    print(p)


## LAB: ## 
### Use the Northwind database
- List the collections
- List the categories collection
- Just display the CategoryID and CategoryName
- Find all the products for Category 1
- Insert a new record into products providing a CategoryID, ProductID = 100 ProductName and UnitPrice
- Display the record
- Update the record you just inserted to increase the price by 1
- Display the record
- Remove the record

<br>
<details><summary>Click for <b>code</b></summary>
<p>

```python
nw = client.Northwind
nw.list_collection_names()
list(nw.categories.find())
list(list(nw.categories.find({},{"CategoryID":1, "CategoryName":1, "_id":0})))
list(nw.products.find({"CategoryID":1}))
nw.products.insert_one({"CategoryID": 1, "ProductID":100, "ProductName":"NewProduct", "UnitPrice": 19})
nw.products.find_one({"ProductID":100})
nw.products.update_one({"ProductID":100}, {"$inc": {"UnitPrice":1}})
nw.products.find_one({"ProductID":100})
nw.products.delete_one({"ProductID":100})
```
</p>
</details>

In [ ]:
nw = client.Northwind
nw.list_collection_names()
nw.list_collection_names()
list(nw.categories.find())
list(list(nw.categories.find({},{"CategoryID":1, "CategoryName":1, "_id":0})))
list(nw.products.find({"CategoryID":1}))
nw.products.insert_one({"CategoryID": 1, "ProductID":100, "ProductName":"NewProduct", "UnitPrice": 19})
nw.products.find_one({"ProductID":100})
r = nw.products.update_one({"ProductID":100}, {"$inc": {"UnitPrice":1}})
print(r.acknowledged)
#dir(r)
nw.products.find_one({"ProductID":100})
# nw.products.delete_one({"ProductID":100})

## Indexes can be used to speed up searches

In [ ]:
db = client.productdb
list(db.products.list_indexes())

## There are a lot of optional parameters for indexes


In [ ]:
help(db.products.create_index)

## Create an index on the manufacturer key in ascending order
<p>
<details><summary>Click for <b>javascript</b></summary>
<p>

```
db.products.createIndex({"manufacturer":1})
db.products.getIndexes()
db.products.findOne({"manufacturer":"KitchenAid"})
db.products.dropIndex("manufacturer_1")
```
</p>
</details>
<p>
<details><summary>Click for <b>java</b></summary>
<p>

```
...
import com.mongodb.client.model.Indexes;
...

DB db = client.getDatabase("productdb");
MongoCollection<Document> products = db.getCollection("products");
products.createIndex(Indexes.ascending("manufacturer"));
products.getIndexes()
products.dropIndex("manufacturer_1")
```
</p>
</details>


In [ ]:
#db.products.create_index("manufacturer")
# print(list(db.products.list_indexes()))
# print(db.products.find_one({"manufacturer":"KitchenAid"}))
db.products.drop_index("manufacturer_1")

## Create a compound index on the `manufacturer` and `model` keys in ascending order
<p>
<details><summary>Click for <b>javascript</b></summary>
<p>

```
db.products.createIndex({"manufacturer":1, "model":1})
db.products.getIndexes()
db.products.findOne({"manufacturer":"KitchenAid"})
db.products.dropIndex("manufacturer_1_model_1")
```
</p>
</details>
<p>
<details><summary>Click for <b>java</b></summary>
<p>

```
...
import com.mongodb.client.model.Indexes;
...

DB db = client.getDatabase("productdb");
MongoCollection<Document> products = db.getCollection("products");
products.createIndex(Indexes.ascending("manufacturer", "model"));
products.getIndexes()
products.dropIndex("manufacturer_1_model_1")
```
</p>
</details>


In [ ]:
#db.products.create_index([("manufacturer", 1), ("model", 1)], name = 'manufacturer_1_model_1')
# print(list(db.products.list_indexes()))
# print(db.products.find_one({"manufacturer":"KitchenAid", "model" : 6}))
db.products.drop_index("manufacturer_1_model_1")


## Create a compound index on the `manufacturer` ascending and `model`  descending
<p>
<details><summary>Click for <b>javascript</b></summary>
<p>

```
db.products.createIndex({"manufacturer":1, "model":-1})
db.products.getIndexes()
db.products.findOne({"manufacturer":"KitchenAid"})
db.products.dropIndex("manufacturer_1_model_-1")
```
</p>
</details>
<p>
<details><summary>Click for <b>java</b></summary>
<p>

```
...
import com.mongodb.client.model.Indexes;
...

DB db = client.getDatabase("productdb");
MongoCollection<Document> products = db.getCollection("products");
// products.createIndex(Indexes.ascending("manufacturer", "model"));
products.createIndex(Indexes.compoundIndex(Indexes.ascending("manufacturer"), Indexes.descending("model")));
products.getIndexes()
products.dropIndex("manufacturer_1_model_1")
```
</p>
</details>




In [ ]:
db.products.create_index([("manufacturer", 1), ("model", -1)], name = 'manufacturer_1_model_-1')
print(list(db.products.list_indexes()))
print(db.products.find_one({"manufacturer":"KitchenAid"}))
db.products.drop_index("manufacturer_1_model_-1")

## Python can also use the enums for ascending and descending instead of 1 or -1, and can also specify an index name

In [ ]:
db.products.create_index([("manufacturer", 1), ("model", pymongo.ASCENDING)], name = 'manufacturer_asc_model_asc')
db.products.create_index([("manufacturer", 1), ("model", pymongo.DESCENDING)], name = 'manufacturer_asc_model_desc')
print(list(db.products.list_indexes()))



## Can also use `hint` to force it to use a specific index

In [ ]:
print(list(db.products.find({"manufacturer":"KitchenAid"}).hint('manufacturer_asc_model_asc')))
print('*' * 20)
print(list(db.products.find({"manufacturer":"KitchenAid"}).hint('manufacturer_asc_model_desc')))
db.products.drop_index("manufacturer_asc_model_asc")
db.products.drop_index("manufacturer_asc_model_desc")

## Text indexes are also possible

In [ ]:
from pymongo import TEXT
db.products.update_one({"manufacturer":"GE"}, {"$set": {"comment": "this product is on backorder"}})
db.products.update_one({"manufacturer":"Samsung"}, {"$set": {"comment": "this product is in stock"}})
db.products.update_one({"manufacturer":"KitchenAid"}, {"$set": {"comment": "backorder product"}})
list(db.products.find())
db.products.create_index([("comment", TEXT)], default_language = 'english')
print(list(db.products.list_indexes()))

## Use the `$text` and `$search` operators with find

In [ ]:
#db.products.drop_index('comment_text')
list(db.products.find({"$text": {"$search": "backorder"}}))


## Sometimes it is desirable to put validators (like SQL constraints) onto a collection to guarantee it meets certain rules. To do this in mongosh it would look like this:
```javascript
db.createCollection("address", {
    validator:{
        firstname: {$type:"string"}
        }})

```

<p>
<details><summary>Click for <b>java</b></summary>
<p>

```
db.createCollection("address", new CreateCollectionOptions()
    .validationOptions(new ValidationOptions()
                .validator(Document.parse("{ \"firstname\" : { \"type\" : \"string\" } }"))));

```
</p>
</details>

## In Python, we would instead use the validator parameter

In [ ]:
db.test.drop()
db.create_collection("test", validator = {
                "firstname" : {"$type":"string"},
                "lastname" : {"$type":"string"}
        })
print(db.list_collection_names())
db.test.options().get('validator')

In [ ]:
#db.test.insert_one({"firstname":"joey", "lastname":"gagliardo"})
#db.test.insert_one({"firstname":"joey", "Lastname":"gagliardo"})
db.test.insert_one({"firstname":"John", "lastname":"gagliardo", 'city':'Miami'})
list(db.test.find())

## Get the current validator and modify it

In [ ]:
email_regex = r'''^[^@\s]+@[^@\s\.]+\.[^@\.\s]+$'''

v = db.test.options().get('validator')
print(v)
v["$or"] =  [ {"phone": {"$type": "string"}}, 
              {"email": {"$regex": email_regex}},
              {"status": {"$in": ["active","inactive"]}}]
print(v)

db.command({"collMod":"test", "validator": v})
db.test.options().get('validator')

## Must have either an email or a phone number or status

In [ ]:
#db.test.insert_one({"firstname":"joey", "lastname":"gagliardo"})

#db.test.insert_one({"firstname":"joey", "lastname":"gagliardo", "email":"joey@me.com"})
#db.test.insert_one({"firstname":"joey", "lastname":"gagliardo", "email":"joey#me.com"})

#db.test.insert_one({"firstname":"joey", "lastname":"gagliardo", "phone":"555-5555"})

#db.test.insert_one({"firstname":"joey", "lastname":"gagliardo", "status": "active"})
#db.test.insert_one({"firstname":"joey", "lastname":"gagliardo", "status": "unknown"})


## Aggregation pipelines are way to aggregate data, but also filter and sort it and more. You build a series of commands you want it to run and then submit that list or pipeline to the aggregate method.

In [ ]:
# The following is a template of what a pipeline would look like
pipeline = [
  { "$match" : { … } },
  { "$group" : { … } },
  { "$sort" : { … } },
  { "$unwind" : { … } },
  { "$lookup" : { … } },
  ...
]
db.collectionName.aggregate(pipeline, options)

## Regular find method

In [ ]:
db = client.Northwind
#db.products.create_index([("ProductName", 1)])

list(db.products.find({"ProductName":"Chang"}))

## The pipeline must be a list of dictionaries that have the operations you want to perform. Here is a simple filter using the `$match` operator.

In [ ]:
pipeline = [{"$match": { "ProductName":"Chang"}}]
list(db.products.aggregate(pipeline))

In [ ]:

list(db.products.aggregate([
  { "$match" : { "CategoryID" : 1 } }
]))

## This example combines a filter and a sort together

In [ ]:
pipeline = [{"$match": { "CategoryID":1}}
           ,{"$sort": {"ProductID": 1}}]
list(db.products.aggregate(pipeline))

## Adding a `$project` operator allows you to include/exclude and rename fields. Note the use of the `$` in front of the field name.

In [ ]:
#list(db.products.find({"CategoryID":1}, {"_id":0, "ProductID":1, "Name":"$ProductName"}).sort("ProductID"))
pipeline = [{"$match": {"CategoryID":1}}
           ,{"$sort": {"ProductID": 1}}
           ,{"$project": {"_id":0, "ProductID":1, "Name": "$ProductName"}}]
list(db.products.aggregate(pipeline))

## Add a limit operator to return a fixed number of documents 

In [ ]:
pipeline1 = [{"$match": { "CategoryID":1}}
           ,{"$sort": {"UnitPrice": -1}}
           ,{"$project": {"_id":0, "ProductID":1, "ProductName":1, "UnitPrice":1}}
           ,{"$limit":5}]
list(db.products.aggregate(pipeline1))

## The group operator is really like a distinct clause in SQL to find all the distinct values for the grouping columns. You must call the column you want to group on _id, but you can rename it later with a project.

In [ ]:
pipeline = [{"$group": { "_id": "$CategoryID"}}]
list(db.products.aggregate(pipeline)) 
# SELECT DISTINCT CategoryID AS _id FROM products
# SELECT CategoryID AS _id FROM products GROUP BY CategoryID


## To add aggregates to the mix, create the alias you want the column to have and the operation to perform. In this example, summing up a one for each document gives a count.

In [ ]:
pipeline = [{"$group": { "_id": "$CategoryID", "product_count": {"$sum":1}}}]
list(db.products.aggregate(pipeline)) 
# SELECT CategoryID AS _id, COUNT(*) AS product_count FROM products GROUP BY CategoryID


## Could use \$sum and \$avg and other aggregate operators

In [ ]:
pipeline = [{"$group": { "_id": "$CategoryID"
                        , "product_count": {"$sum":1}
                        , "product_total": {"$sum":"$UnitPrice"}
                        , "product_avg": {"$avg":"$UnitPrice"}
                       }
            }]
list(db.products.aggregate(pipeline)) 
# SELECT CategoryID AS _id
# , COUNT(*) AS product_count
# , SUM(UnitPrice) AS product_total
# , AVG(UnitPrice) AS product_avg
# FROM products GROUP BY CategoryID


## Doing a $match after grouping is like a HAVING clause

In [ ]:
# Make constants instead of using the `$` operators and can call them whatever you like to make it more SQL like
GROUP = "$group"
FILTER = "$match"
HAVING = "$match"
SELECT = "$project"
pipeline = [{GROUP: { "_id": "$CategoryID"
                        , "product_count": {"$sum":1}
                        , "product_total": {"$sum":"$UnitPrice"}
                        , "product_avg": {"$avg":"$UnitPrice"}
                       }
            }
            , {HAVING: {"product_total": {"$gte":10}}}
            , {SELECT: {"CategoryID":"$_id", "product_avg":1, "product_count":1, "product_total":1, "_id":0}}
           ]
list(db.products.aggregate(pipeline)) 
# SELECT CategoryID AS _id
# , COUNT(*) AS product_count
# , SUM(UnitPrice) AS product_total
# , AVG(UnitPrice) AS product_avg
# FROM products GROUP BY CategoryID
# HAVING COUNT(*) >= 10


## The `$push` operator can be used to create a list of nested children items to get a nested repeating effect similar to the Hive `COLLECT_LIST` function

In [ ]:
pipeline = [{"$group": { "_id": "$CategoryID"
                        , "product_count": {"$sum": 1}
                        , "product_avg": {"$avg": "$UnitPrice"}
                        , "product_list": {"$push": "$ProductName" }
                       }
            }
            , {"$project": {"CategoryID":"$_id", "product_count":1, "product_avg":1, "product_list":1, "_id":0}}
           ]
                        
list(db.products.aggregate(pipeline)) 

# MySQL GROUP_CONCAT  rows A B C    --> "A, B, C"
# Hive COLLECT_LIST   rows A B C    --> ["A", "B", "C"]
# SELECT CategoryID, COUNT(*) AS product_count
# , AVG(UnitPrice) AS product_avg, COLLECT_LIST(ProductName)
# FROM Products
# GROUP BY CategoryID

# BigQuery ARRAY_AGG()

## You can also collect a list of multiple fields together to create a list of structures

In [ ]:
pipeline = [{"$group": { "_id": "$CategoryID"
                        ,"Products" : { "$push": {"ProductID": "$ProductID"
                                                  ,"ProductName": "$ProductName"
                                                  , "UnitPrice": "$UnitPrice"}}
                        }
             }
            ,{"$project": {"CategoryID": "$_id", "Products":1, "_id":0}}
            ]
                        
list(db.products.aggregate(pipeline)) 


## Here, we combine many elements together

In [ ]:
pipeline = [{"$group": { "_id": "$CategoryID"
                        ,"product_list" : { "$push": {"ProductID": "$ProductID"
                                                  ,"ProductName": "$ProductName"
                                                  , "UnitPrice": "$UnitPrice"}}
                        , "product_count": {"$sum": 1}
                        , "product_avg": {"$avg": "$UnitPrice"}
                        }
             }
            , {"$project": {"CategoryID": "$_id", "product_count": 1, "product_avg": 1, "product_list":1, "_id":0}}
            , {"$match": {"product_count": {"$gte":10}}}
            , {"$sort": {"product_count": -1, "product_avg":1}}
            ]
   
x = db.products.aggregate(pipeline)
list(x)


## Let's save the results of the nested outcome to a collection

In [ ]:
pipeline = [{"$group": { "_id": "$CategoryID"
                        ,"product_list" : { "$push": {"ProductID": "$ProductID"
                                                  ,"ProductName": "$ProductName"
                                                  , "UnitPrice": "$UnitPrice"}}
                        , "product_count": {"$sum": 1}
                        , "product_avg": {"$avg": "$UnitPrice"}
                        }
             }
            , {"$project": {"CategoryID": "$_id", "product_count": 1, "product_avg": 1, "product_list":1, "_id":0}}
            , {"$match": {"product_count": {"$gte":10}}}
            , {"$sort": {"product_count": -1, "product_avg":1}}
            ]

# get rid of what may have been there from before
db.products_nested.drop()

# get the results back to the client
x = db.products.aggregate(pipeline)

# save the results to a collection
db.products_nested.insert_many(x)
list(db.products_nested.find())

## A better option is to use the `$out` operator in the pipeline to let the Mongo API directly do it in one step

In [ ]:
db.products_nested.drop()
pipeline.append({"$out": {"db": "Northwind", "coll": "products_nested"}})
print(pipeline)

db.products.aggregate(pipeline)
list(db.products_nested.find())

## Suppose you have some nested data and want to flatten or unnest it back to a traditional denormalized format, use the `$unwind` operator

In [ ]:
pipeline = [{"$unwind": "$product_list"}]
x = db.products_nested.aggregate(pipeline)
list(x)


## The results are a bit funny looking, so use `$project` to fix them up

In [ ]:
pipeline = [{"$unwind": "$product_list"}
           ,{"$project": {"CategoryID":1, "product_avg":1, "product_count":1
                          , "ProductID":"$product_list.ProductID"
                          , "ProductName":"$product_list.ProductName"
                          , "UnitPrice":"$product_list.UnitPrice"
                          , "_id":0}}]
x = db.products_nested.aggregate(pipeline)
list(x)

# LATERAL VIEW EXPLODED

## Generally, a MongoDB database should not rely on relational structure but instead each document should contain everything it needs, but if you need to, you can use a `$lookup` operator which is sort of like using a `JOIN` or correlated subquery in SQL. 
<br>
`
SELECT ProductID, ProductName, CategoryID
       , (SELECT CategoryName 
       FROM Categories AS c WHERE c.CategoryID = p.CategoryID) AS CategoryName
 FROM Products AS p`
<br>
<br>
`
SELECT p.ProductID, p.ProductName, p.CategoryID, c.CategoryName
 FROM Products AS p
 LEFT JOIN Categories AS c ON p.CategoryID = c.CategoryID`
<br>
##Except the looked up collection embeds all the documents as a list into the outer table. It's the same as we would get from the `$push` operation.

In [ ]:
pipeline = [{"$lookup": {"from": "products"
                        , "localField": "CategoryID"
                        , "foreignField": "CategoryID"
                        , "as": "Products"}}
            ]
x = db.categories.aggregate(pipeline)
list(x)


## If we want to flatten it out to look like the normal result of a SQL JOIN, use `$unwind` and `$project`.

In [ ]:
pipeline = [{"$lookup": {"from": "products"
                        , "localField": "CategoryID"
                        , "foreignField": "CategoryID"
                        , "as": "Products"}}
            , {"$unwind": "$Products"}
            , {"$project": {"_id":0, "CategoryID":1, "CategoryName":1
                            , "ProductID":"$Products.ProductID"
                            , "ProductName": "$Products.ProductName"
                            , "UnitPrice": "$Products.UnitPrice"
                           }}
            ]
x = db.categories.aggregate(pipeline)
list(x)

## If we do it in the opposite order, this time from the child to the parent, even though there is only one category per product, it still returns the single category document in an array.

In [ ]:
pipeline = [{"$lookup": {"from": "categories"
                        , "localField": "CategoryID"
                        , "foreignField": "CategoryID"
                        , "as": "Category"}}
            ]
x = db.products.aggregate(pipeline)
list(x)



## Could use `$unwind` and `$project` to make it normal looking, but if there are more than one categories for a product it would repeat the product record twice. 

In [ ]:
pipeline = [{"$lookup": {"from": "categories"
                        , "localField": "CategoryID"
                        , "foreignField": "CategoryID"
                        , "as": "Categories"}}
            ,{"$unwind": "$Categories"}
           ,{"$project": {"CategoryID": "$Categories.CategoryID"
                          , "CategoryName": "$Categories.CategoryName"
                          , "ProductID":1, "ProductName":1, "UnitPrice":1, "_id":0}}
            ]
x = db.products.aggregate(pipeline)
list(x)


## So an alterate logic would be to just pick the first record out of the collection using  `$project` to fix it up. Here we use two `$project` clauses to first get the list of just CategoryNames, then get only the first element of that

In [ ]:
pipeline = [{"$lookup": {"from": "categories"
                        , "localField": "CategoryID"
                        , "foreignField": "CategoryID"
                        , "as": "Categories"}}
            ,{"$project": {"CategoryID":1, "CategoryNames":"$Categories.CategoryName", "ProductID":1, "ProductName":1, "UnitPrice":1}}
            ,{"$project": {"_id":0, "CategoryID":1, "Name": {"$arrayElemAt": ["$CategoryNames", 0] }, "ProductID":1, "ProductName":1, "UnitPrice":1}}
            ]
x = db.products.aggregate(pipeline)
list(x)


## Instead of using two `$projects` we could combine it into one more complicated one also

In [ ]:
pipeline = [{"$lookup": {"from": "categories"
                        , "localField": "CategoryID"
                        , "foreignField": "CategoryID"
                        , "as": "Categories"}}
            ,{"$project": {"CategoryID":1, "CategoryName":{"$arrayElemAt": ["$Categories.CategoryName", 0] }, "ProductID":1, "ProductName":1, "UnitPrice":1, "_id":0}}
            ]
x = db.products.aggregate(pipeline)
list(x)


In [ ]:
db = client['Northwind']
db.list_collection_names()


## LAB: ## 
### Use the Northwind database
- Group the customers collection by country
- Show the total number of customers in each country and a list of those customers
- Only show countries with at least two customers
- Sort the list in descending order by count and ascending by country name

<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Use \$group \$project \$match \$sort in an aggregate pipeline
<br>
Use \$sum to get total customers per country
<br>
Use \$push to create a list of CustomerID and CompanyName
<br>
</p>
</details>
<br>
<details><summary>Click for <b>code</b></summary>
<p>

```python
pipeline = [
    {"$group": {"_id": "$Country"
              , "customer_count": {"$sum": 1}
              , "Customers": {"$push":  {"CustomerID": "$CustomerID", "CompanyName": "$CompanyName"}}
                      }}
    ,{"$project": {"_id":0, "Country":"$_id", "customer_count":1, "Customers":1}}
    ,{"$match": {"customer_count": {"$gt":1}}}
    ,{"$sort": {"customer_count": -1, "Country":1}}    
           ]
list(db.customers.aggregate(pipeline))
```
</p>
</details>